In [1]:
import sys

sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\open_ai_assistant_v2")

from src.tools.vanna.vanna_tool import VannaTool

vn = VannaTool()

In [2]:
from src.utils.excel_data_reader import get_excel_data

archivo_excel = "../../assets/Diccionario_datos.xlsx"
columnas_hoja_2 = ["ddl", "about_variables", "about_database", "questions", "answers"]
data = get_excel_data(archivo_excel, columnas_hoja_2, "Vanna")

In [5]:
# Para alimentar la db
# Demora como 2 min

for ddl in data["ddl"]:
    vn.feed_ddl(ddl=ddl)

for documentation in data["about_variables"]:
    vn.feed_documentation(documentation=documentation)

for documentation in data["about_database"]:
    vn.feed_documentation(documentation=documentation)

for example in data["examples"]:
    q = example["question"]
    sql = example["answer"]
    vn.feed_examples(question=q, sql=sql)

Adding ddl: CREATE TABLE IF NOT EXISTS dbo_v2.fcs_computadores (
    Id INT PRIMARY KEY,
    IP VARCHAR(100),
    Puerto INT,
    Puerto_Secundario INT,
    Id_Modbus INT,
    Id_Modbus_Secundario INT,
    Tag VARCHAR(100),
    Compatibilidad_Modicon VARCHAR(100),
    Id_Computador_Redundante INT,
    Estado VARCHAR(100),
    Master VARCHAR(100),
    Usuario VARCHAR(100),
    Contraseña VARCHAR(100),
    Leer_Tiempo_Real VARCHAR(100),
    Leer_Configuracion VARCHAR(100),
    Leer_Alarmas VARCHAR(100),
    Leer_Eventos VARCHAR(100),
    Leer_Historicos VARCHAR(100),
    Grupo_Destino VARCHAR(100),
    Unidad_Destino VARCHAR(100),
    Grupo_Fuente VARCHAR(100),
    Unidad_Fuente VARCHAR(100),
    Orden_Archivos INT,
    Numero_Maximo_Horarios INT,
    Numero_Maximo_Diarios INT,
    Numero_Maximo_Proves INT,
    Numero_Maximo_Batch INT,
    Tipo_Protocolo VARCHAR(100),
    Tiempo_Proceso_Historico INT,
    Tiempo_Proceso_TiempoReal INT,
    IdFirmware_fk INT,
    IdEquipo_fk INT,
    Serv

In [38]:
# Para alimentar la db con un ejemplo 
q = "Request for a list of flow computers"
sql = """
SELECT c.Id, Tag, f.Firmware as 'Revision/Version', tc.Nombre as Marca
FROM dbo_v2.fcs_computadores c 
JOIN dbo_v2.fcs_firmware f ON f.Id = c.IdFirmware_fk
JOIN dbo_v2.fcs_tipo_computador tc ON tc.Id = f.IdTipo_Computador_fk;
"""
vn.feed_examples(question=q, sql=sql)

In [18]:
documentation="When user asks about a question and DDLs are no related to the question and also there are no examples, answer with 'No SELECT statement could be found in the SQL code'"
vn.feed_documentation(documentation)

Adding documentation....


In [10]:
training_data = vn.get_training_data()
print(training_data)

             id training_data_type question  \
0    243671-doc      documentation     None   
1    243722-doc      documentation     None   
2    243669-doc      documentation     None   
3    243663-doc      documentation     None   
4    243756-doc      documentation     None   
..          ...                ...      ...   
116  243721-doc      documentation     None   
117  243755-doc      documentation     None   
118  243676-doc      documentation     None   
119  243727-doc      documentation     None   
120  243718-doc      documentation     None   

                                               content  
0    When user asks about 'i-pentane mole' use 'Mol...  
1    When user asks about 'octane mole' use 'Mol (%...  
2    When user asks about 'ethane mole' use 'Mol (%...  
3    When user asks about 'Gross flow' use 'Vazão B...  
4    When user asks about 'Z base' use 'Z base' in ...  
..                                                 ...  
116  When user asks about 'Pipe Diam

In [3]:
# Para borrar todos los datos de la db
# Demora como 2 min
vn.remove_all_data()

In [11]:
q = "When user asks about a question and DDLs are no related to the question and also there are no examples, answer with 'No SELECT statement could be found in the SQL code'"
for index, data in enumerate(training_data["id"]):
    if training_data["content"][index] == q:
        print(training_data["content"][index])
        print(training_data["id"][index])

When user asks about a question and DDLs are no related to the question and also there are no examples, answer with 'No SELECT statement could be found in the SQL code'
243768-doc


In [12]:

# Para borrar un elemento de la db
vn.remove_single_data("243768-doc")


"No SELECT statement could be found in the SQL code"

In [14]:
question = "list of flow computers"
generated_sql = vn.generate_sql(question)
print(generated_sql)

SELECT c.Id, c.Tag, f.Firmware AS 'Revision/Version', tc.Nombre AS Marca
FROM dbo_v2.fcs_computadores c 
JOIN dbo_v2.fcs_firmware f ON f.Id = c.IdFirmware_fk
JOIN dbo_v2.fcs_tipo_computador tc ON tc.Id = f.IdTipo_Computador_fk;


In [5]:
res = vn.run_sql(generated_sql)
print(res)

C:\Users\lauth\OneDrive\Desktop\open_ai_assistant_v2\src\tools\vanna\vanna_tool.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, self.conn)


DatabaseError: Execution failed on sql 'SELECT Id, Tag, Modelo, Marca, Fabricante, Revision
FROM dbo_v2.fcs_computadores;': ('42S22', "[42S22] [Microsoft][ODBC Driver 11 for SQL Server][SQL Server]Invalid column name 'Modelo'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC Driver 11 for SQL Server][SQL Server]Invalid column name 'Marca'. (207); [42S22] [Microsoft][ODBC Driver 11 for SQL Server][SQL Server]Invalid column name 'Fabricante'. (207); [42S22] [Microsoft][ODBC Driver 11 for SQL Server][SQL Server]Invalid column name 'Revision'. (207)")